# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [3]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
# Your code here
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train, y_train)

LinearRegression()

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [6]:
# Your code here
from sklearn.metrics import mean_squared_error

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

Train Mean Squarred Error: 0.16025695964655187
Test Mean Squarred Error: 0.17595331097301362


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [7]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    df = pd.DataFrame(data)
    outlist = []
    l = len(df)
    fold_len = l // k 
    extra = l - fold_len * k
    for i in range(k):
        if i < extra:
            dfi = df[i*fold_len:((i+1)*fold_len + 1)]
        else:
            dfi = df[i*fold_len:((i+1)*fold_len)]
        outlist.append(dfi)
    
    # add 1 to fold size to account for leftovers           
    return outlist
#ames_data = kfolds(ames, 10)
# for i in range(10):
#     display(i, len(ames_data[i]),ames_data[i])


### Apply it to the Ames Housing data

In [8]:
# Make sure to concatenate the data again
ames_data = X_train
ames_data['SalePrice_log'] = y_train
#wtf does this mean?
#ames_data
# display(X_train)
# display(y_train)

<ipython-input-8-6c78b05b25e9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ames_data['SalePrice_log'] = y_train


In [11]:
# Apply kfolds() to ames_data with 5 folds
ames_data_folds = kfolds(ames_data, 5)
ames_data_folds[0]

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
254,-0.144655,0.545525,-0.260660,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-0.349383
1066,-0.278727,-1.020176,0.274901,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.163944
638,-0.059809,-1.032015,-1.763377,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1.686422
799,-0.442560,-0.374308,0.629080,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0.121392
380,-1.147255,-0.233149,0.495580,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,-0.681205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,-2.527426,-1.161154,0.378065,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,-0.198630
544,1.229552,-1.602679,0.128244,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.187253
1376,-0.703073,-1.055828,-1.786061,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,-1.515667
660,0.605515,0.103972,0.899679,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.429254


### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [23]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    preproc_k = ames_data_folds[n]
    preprocksplit = kfolds(preproc_k,k)
    #display(preprocksplit[n])
    #display('lenpps', len(preprocksplit))
    currtrain = pd.DataFrame()
    currtest = pd.DataFrame()
    for split in range(k):
        #print(n,split)
        if n == split:
            currtest = preprocksplit[split]
        else:
            currtrain = currtrain.append(preprocksplit[split])
    train = currtrain
    test = currtest
    #display('train', train)
    #display('test', test)
    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']
    X_test = test.drop('SalePrice_log', axis=1)
    y_test = test['SalePrice_log']
    # Fit a linear regression model
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_mse = mean_squared_error(y_train, y_hat_train)
    test_mse = mean_squared_error(y_test, y_hat_test)
    test_errs.append(test_mse)
    train_errs.append(train_mse)

print(train_errs)
print(test_errs)
'''woohoo!'''

[0.11304944541300557, 0.11999362893540465, 0.17388177548822242, 0.10616384372668167, 0.0918763084933603]
[0.22664344146577517, 0.18425455963177878, 0.38278939963577896, 0.26188973161291357, 0.15111805848002088]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [27]:
# Your code here
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score


cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error')

array([-0.12431546, -0.19350065, -0.1891053 , -0.17079325, -0.20742705])

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [28]:
# Your code here
cv_5_results  = np.mean(cross_val_score(linreg, X, y, cv=5,  scoring='neg_mean_squared_error'))
display(cv_5_results)

-0.17702834210001125

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 